### Effect of COVID-19 on the water quality of India

In [13]:
#import needed libraries
from os import path as op
import pickle
import fiona
import geemap
import ipyleaflet
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium


In [14]:
#!pip install geehydro
import geehydro
import folium

In [15]:
ee.Initialize()

In [16]:
c = fiona.open('/home/jovyan/Analysis/Impacts-of-COVID-19/India_water/IND_water_areas_dcw.shp') #water shapefile data of India

polygons = []

for row in c:
  polygons.append(row['geometry']['coordinates'])

In [17]:
#Tapi river
aoi = ee.Geometry.Polygon(
         [[[73.54407919861856, 21.506806808887394],
          [73.54407919861856, 21.165269300114176],
          [74.040523900767, 21.165269300114176],
          [74.040523900767, 21.506806808887394]]])

In [18]:
#function to evaluate the Normalised Difference Water Index (NDWI)
def ndwi_f(img):
  ndwi=img.normalizedDifference(['B3','B8']).rename('NDWI')
  return img.addBands(ndwi).updateMask(ndwi.gt(0))

In [19]:
#function to evaluate the Normalised Difference Chlorophyll Index (NDCI)
def ndci_f(img):
  #water mask
  ndci = img.normalizedDifference(['B5', 'B4']).rename('NDCI');
  return img.addBands(ndci)


In [20]:
band_sel = [ 'B3', 'B4', 'B5', 'B8'] #select bands to use
#helper functions
def map_(start_date,end_date):
  IMG=ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(aoi).filterDate(start_date,end_date).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).median().clip(aoi)
  IMG_water = ndwi_f(IMG)
  IMG_NDCI = ndci_f(IMG_water)
  Map=folium.Map(location=[   21.33,73.79 ],zoom_start=10)

  viz = {'min':0.02,'max':0.05,'palette':['cyan','orange','red']}

  Map.addLayer(IMG.clip(aoi),{'bands':['B4','B3','B2'],'min':0,'max':3500},'IMG')
  Map.addLayer(IMG_water.clip(aoi).select('NDWI'),{'palette':['cyan']},"IMG_water")
  Map.addLayer(IMG_NDCI.clip(aoi).select('NDCI'),viz,"IMG_NDCI")
  Map.setControlVisibility(layerControl=True, fullscreenControl=False, latLngPopup=True)
  return Map

In [33]:
#obtain data for these months
january = map_('2020-01-01','2020-01-30')
march=map_('2020-03-01','2020-03-30')
april=map_('2020-04-01','2020-04-30')
may=map_('2020-05-01','2020-05-30')
dec=map_('2020-10-01','2020-12-30') #december



In [26]:
type(march)

folium.folium.Map

In [34]:
january

In [29]:

march

In [30]:
april

In [31]:
may


In [32]:
dec